<h2>Data Cleaning and preprocessing of the CSV file</h2>

In [9]:
import numpy as np 
import pandas as pd
from datetime import date as dated
nike_data = pd.read_csv('Nike_Sales_Uncleaned.csv')

np.random.seed(0)

<h4>#checking the no of null valued cells in each column</h4>

In [10]:

missing_values = nike_data.isnull().sum()
missing_values

Order_ID               0
Gender_Category        0
Product_Line           0
Product_Name           0
Size                 510
Units_Sold          1235
MRP                 1254
Discount_Applied    1668
Revenue                0
Order_Date           616
Sales_Channel          0
Region                 0
Profit                 0
dtype: int64

We can see that columns Units_Sold, Discount_applied 
    and MRP have the highest amount 
of null values almost upto half of the size so here we remove these columns from the dataset

In [11]:


nike_data.drop(['MRP'],axis=1,inplace=True)
nike_data.drop('Discount_Applied',axis=1,inplace=True,errors='ignore')
nike_data.drop('Revenue',axis=1,inplace=True,errors='ignore')
nike_data.drop('Units_Sold',axis=1,inplace=True,errors='ignore')


#checking the no of null valued cells in each column

In [12]:

missing_values = nike_data.isnull().sum()
missing_values

Order_ID             0
Gender_Category      0
Product_Line         0
Product_Name         0
Size               510
Order_Date         616
Sales_Channel        0
Region               0
Profit               0
dtype: int64

We now fill null valued cells with the value 'unrecorded' instead of dropping null valued cells preventing data loss

In [14]:

nike_data['Size'].fillna('Unrecorded',inplace=True)

print(nike_data)

      Order_ID Gender_Category Product_Line      Product_Name        Size  \
0         2000            Kids     Training       SuperRep Go           M   
1         2001           Women       Soccer     Tiempo Legend           M   
2         2002           Women       Soccer       Premier III           M   
3         2003            Kids    Lifestyle        Blazer Mid           L   
4         2004            Kids      Running    React Infinity          XL   
...        ...             ...          ...               ...         ...   
2495      4495            Kids   Basketball     Kyrie Flytrap          XL   
2496      4496             Men   Basketball     Kyrie Flytrap           L   
2497      4497             Men       Soccer     Tiempo Legend           7   
2498      4498           Women     Training  ZoomX Invincible  Unrecorded   
2499      4499           Women      Running          Air Zoom           M   

      Order_Date Sales_Channel     Region   Profit  
0     2024-03-09      

In [16]:
print(nike_data.head())

   Order_ID Gender_Category Product_Line    Product_Name Size  Order_Date  \
0      2000            Kids     Training     SuperRep Go    M  2024-03-09   
1      2001           Women       Soccer   Tiempo Legend    M  2024-07-09   
2      2002           Women       Soccer     Premier III    M         NaN   
3      2003            Kids    Lifestyle      Blazer Mid    L  04-10-2024   
4      2004            Kids      Running  React Infinity   XL  2024/09/12   

  Sales_Channel     Region   Profit  
0        Online  bengaluru  -770.45  
1        Retail        Hyd  -112.53  
2        Retail     Mumbai  3337.34  
3        Online       Pune  3376.85  
4        Retail      Delhi   187.89  


#Here we can see the order_date column is very messy and the all the dates present are not in the same format

#we bring here all the different date formats into one common date format with the following function:

In [17]:

def convert_to_date01(str1):
    if pd.isna(str1) or str1=='':
        return dated(2000,1,1)
    for delim in ['/',',','-']:
        if delim in str1:
            parts = str1.strip().split(delim)
            break
    else:
        return dated(2000,1,1)
    parts = [int(p) for p in parts if p.isdigit()]
    dt=mt=yr=0
    for p in parts:
        if p>1980:
            yr=p
        elif p<12 and mt==0:
            mt=p
        else:
            dt=p
    if yr==0: yr=2000
    if dt==0: dt=1
    if mt==0: mt=1    
    return dated(yr,mt,dt)

nike_data['Order_Date'] = nike_data['Order_Date'].apply(convert_to_date01)
nike_data.head()

,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Order_Date,Sales_Channel,Region,Profit
0,2000,Kids,Training,SuperRep Go,M,2024-03-09,Online,bengaluru,-770.45
1,2001,Women,Soccer,Tiempo Legend,M,2024-07-09,Retail,Hyd,-112.53
2,2002,Women,Soccer,Premier III,M,2000-01-01,Retail,Mumbai,3337.34
3,2003,Kids,Lifestyle,Blazer Mid,L,2024-04-10,Online,Pune,3376.85
4,2004,Kids,Running,React Infinity,XL,2024-09-12,Retail,Delhi,187.89


In [22]:
nike_data.groupby('Region').count()

,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Order_Date,Sales_Channel,Profit
Region,,,,,,,,
Bangalore,216,216,216,216,216,216,216,216
Delhi,438,438,438,438,438,438,438,438
Hyd,136,136,136,136,136,136,136,136
Hyderabad,126,126,126,126,126,126,126,126
Kolkata,417,417,417,417,417,417,417,417
Mumbai,418,418,418,418,418,418,418,418
Pune,388,388,388,388,388,388,388,388
bengaluru,220,220,220,220,220,220,220,220
hyderbad,141,141,141,141,141,141,141,141


<h4>Here the Region column contains multiple Sting instances of the same City like Banglore, bengaluru and Hyderabad, hyderabad, hyd and so on
<br>
<br>We now replace the values bengaluru with Banglore and hyderabad, Hyd with Hyderabad for data consistency</h4>

In [23]:

def Cleaning_Regions(region):
    if region == 'bengaluru' or region=='bangaluru':
        return 'Bangalore'
    elif region == 'Hyd' or region == 'hyderbad':
        return 'Hyderabad'
    else:
        return region

nike_data['Region'] = nike_data['Region'].apply(Cleaning_Regions)
nike_data.groupby(['Region']).count()

,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Order_Date,Sales_Channel,Profit
Region,,,,,,,,
Bangalore,436,436,436,436,436,436,436,436
Delhi,438,438,438,438,438,438,438,438
Hyderabad,403,403,403,403,403,403,403,403
Kolkata,417,417,417,417,417,417,417,417
Mumbai,418,418,418,418,418,418,418,418
Pune,388,388,388,388,388,388,388,388


<h4>Checking the no of null valued cells in each column</h4><br>

In [24]:

missing_values = nike_data.isnull().sum()
missing_values

Order_ID           0
Gender_Category    0
Product_Line       0
Product_Name       0
Size               0
Order_Date         0
Sales_Channel      0
Region             0
Profit             0
dtype: int64

In [27]:
nike_data

,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Order_Date,Sales_Channel,Region,Profit
0,2000,Kids,Training,SuperRep Go,M,2024-03-09,Online,Bangalore,-770.45
1,2001,Women,Soccer,Tiempo Legend,M,2024-07-09,Retail,Hyderabad,-112.53
2,2002,Women,Soccer,Premier III,M,2000-01-01,Retail,Mumbai,3337.34
3,2003,Kids,Lifestyle,Blazer Mid,L,2024-04-10,Online,Pune,3376.85
4,2004,Kids,Running,React Infinity,XL,2024-09-12,Retail,Delhi,187.89
...,...,...,...,...,...,...,...,...,...
2495,4495,Kids,Basketball,Kyrie Flytrap,XL,2025-05-14,Online,Pune,2.97
2496,4496,Men,Basketball,Kyrie Flytrap,L,2000-01-01,Online,Hyderabad,-487.36
2497,4497,Men,Soccer,Tiempo Legend,7,2025-02-07,Retail,Bangalore,-918.14
2498,4498,Women,Training,ZoomX Invincible,Unrecorded,2024-11-12,Online,Mumbai,3352.29


<h4>So Now we can see that there are no null valued cells and We have been able to successfully Clean the data available without reducing the row size with minimal Data loss</h4>

We now save the cleaned data set into a new csv file

In [31]:
# Save cleaned data to NEW CSV
nike_data.to_csv('nike_sales_cleaned.csv', index=False)

print("Cleaned dataset saved to 'nike_sales_cleaned.csv'")

Cleaned dataset saved to 'nike_sales_cleaned.csv'
